In [1]:
import pandas as pd, numpy as np
from db_queries import get_ids, get_outputs
import sys, os.path
sys.path.append(os.path.abspath("../.."))

!whoami
!date

ndbs
Thu Nov 19 22:23:42 PST 2020


In [2]:
from pre_processing.id_helper import *

# Goal: Use `get_outputs` to pull DALYs and birth prevalence for Neural Tube Defects from GBD 2017

# 1. Look up id's that I need

In [3]:
search_id_table('cause', 'Neural Tube Defects', case=False)

,cause_id,cause_name,acause,cause_description
348,642,Neural tube defects,cong_neural,None


In [4]:
search_id_table('measure', 'DALYs')

,measure_id,measure_name
1,2,DALYs (Disability-Adjusted Life Years)


In [5]:
get_ids('metric')

,metric_id,metric_name,metric_description
0,1,Number,Like number of deaths
1,2,Percent,Pc is percent of deaths or attributable risk f...
2,3,Rate,Rate
3,4,Rank,Ranking in GBD Compare.
4,5,Years,For those that are measured in years
5,6,p-value,Draw-level statistical significance. (number o...
6,7,MDG p-value,Draw-level statistical significance of achievi...
7,8,Probability of death,Probability of death
8,9,Index score,index score (0-100)


In [6]:
names_to_ids('location', 'India', 'Ethiopia', 'Nigeria')

location_name
India       163
Ethiopia    179
Nigeria     214
Name: location_id, dtype: int64

In [7]:
names_to_ids('age_group', 'Under 5', 'Early Neonatal', 'Late Neonatal', 'Post Neonatal', '1 to 4')

age_group_name
Under 5           1
Early Neonatal    2
Late Neonatal     3
Post Neonatal     4
1 to 4            5
Name: age_group_id, dtype: int64

# 2. Pull DALYs

In [8]:
dalys = get_outputs(
    'cause',
    cause_id = list_ids('cause', 'Neural tube defects'),
    measure_id = list_ids('measure', 'DALYs (Disability-Adjusted Life Years)'),
    metric_id = list_ids('metric', 'Number', 'Rate'),
    location_id = list_ids('location', 'India', 'Ethiopia', 'Nigeria'),
    age_group_id = list_ids('age_group', 'Under 5', 'Early Neonatal', 'Late Neonatal', 'Post Neonatal', '1 to 4'),
    gbd_round_id = list_ids('gbd_round', '2017'),
    year_id = 2017,
)
dalys.head()

,age_group_id,cause_id,location_id,measure_id,metric_id,sex_id,year_id,acause,age_group_name,cause_name,expected,location_name,location_type,measure_name,metric_name,sex,val,upper,lower
0,1,642,163,2,1,3,2017,cong_neural,Under 5,Neural tube defects,False,India,admin0,DALYs (Disability-Adjusted Life Years),Number,Both,44936.678983,61226.307877,32247.186071
1,2,642,163,2,1,3,2017,cong_neural,Early Neonatal,Neural tube defects,False,India,admin0,DALYs (Disability-Adjusted Life Years),Number,Both,14185.414021,23900.946935,8044.466831
2,3,642,163,2,1,3,2017,cong_neural,Late Neonatal,Neural tube defects,False,India,admin0,DALYs (Disability-Adjusted Life Years),Number,Both,5392.882027,8773.937460,3530.200498
3,4,642,163,2,1,3,2017,cong_neural,Post Neonatal,Neural tube defects,False,India,admin0,DALYs (Disability-Adjusted Life Years),Number,Both,8478.122371,12004.392068,6376.735792
4,5,642,163,2,1,3,2017,cong_neural,1 to 4,Neural tube defects,False,India,admin0,DALYs (Disability-Adjusted Life Years),Number,Both,16880.260564,22452.841899,11706.275501


In [9]:
dalys.shape

(30, 19)

In [10]:
columns_to_keep = ['cause_name', 'location_name', 'age_group_name',
                   'measure_name', 'metric_name', 'val', 'upper', 'lower']
ntd_dalys = dalys[columns_to_keep]
ntd_dalys.head()

,cause_name,location_name,age_group_name,measure_name,metric_name,val,upper,lower
0,Neural tube defects,India,Under 5,DALYs (Disability-Adjusted Life Years),Number,44936.678983,61226.307877,32247.186071
1,Neural tube defects,India,Early Neonatal,DALYs (Disability-Adjusted Life Years),Number,14185.414021,23900.946935,8044.466831
2,Neural tube defects,India,Late Neonatal,DALYs (Disability-Adjusted Life Years),Number,5392.882027,8773.937460,3530.200498
3,Neural tube defects,India,Post Neonatal,DALYs (Disability-Adjusted Life Years),Number,8478.122371,12004.392068,6376.735792
4,Neural tube defects,India,1 to 4,DALYs (Disability-Adjusted Life Years),Number,16880.260564,22452.841899,11706.275501


In [11]:
ntd_dalys.query('location_name == "Nigeria"')

,cause_name,location_name,age_group_name,measure_name,metric_name,val,upper,lower
10,Neural tube defects,Nigeria,Under 5,DALYs (Disability-Adjusted Life Years),Number,634154.400036,1.190242e+06,337333.474548
11,Neural tube defects,Nigeria,Early Neonatal,DALYs (Disability-Adjusted Life Years),Number,249737.515909,4.660432e+05,133097.428914
12,Neural tube defects,Nigeria,Late Neonatal,DALYs (Disability-Adjusted Life Years),Number,53939.776440,1.094386e+05,24171.797495
13,Neural tube defects,Nigeria,Post Neonatal,DALYs (Disability-Adjusted Life Years),Number,149943.358987,3.150526e+05,57415.345632
14,Neural tube defects,Nigeria,1 to 4,DALYs (Disability-Adjusted Life Years),Number,180533.748701,4.279059e+05,61141.363578
25,Neural tube defects,Nigeria,Under 5,DALYs (Disability-Adjusted Life Years),Rate,0.018411,3.455509e-02,0.009793
26,Neural tube defects,Nigeria,Early Neonatal,DALYs (Disability-Adjusted Life Years),Rate,1.689435,3.152709e+00,0.900383
27,Neural tube defects,Nigeria,Late Neonatal,DALYs (Disability-Adjusted Life Years),Rate,0.123885,2.513516e-01,0.055516
28,Neural tube defects,Nigeria,Post Neonatal,DALYs (Disability-Adjusted Life Years),Rate,0.022059,4.634926e-02,0.008447
29,Neural tube defects,Nigeria,1 to 4,DALYs (Disability-Adjusted Life Years),Rate,0.006671,1.581078e-02,0.002259


# 3. Pull birth prevalence

In [12]:
ids_to_names('age_group', 164) # What is the 'at birth' age group called (id 164)?

age_group_id
164    Birth
Name: age_group_name, dtype: object

In [13]:
names_to_ids('age_group', 'Birth')

age_group_name
Birth    164
Name: age_group_id, dtype: int64

In [14]:
bp = get_outputs(
    'cause',
    cause_id = list_ids('cause', 'Neural tube defects'),
    measure_id = list_ids('measure', 'Incidence'),
    metric_id = list_ids('metric', 'Number', 'Rate'),
    location_id = list_ids('location', 'India', 'Ethiopia', 'Nigeria'),
    age_group_id = list_ids('age_group', 'Birth'),
    gbd_round_id = list_ids('gbd_round', '2017'),
    year_id = 2017,
)
bp

,age_group_id,cause_id,location_id,measure_id,metric_id,sex_id,year_id,acause,age_group_name,cause_name,expected,location_name,location_type,measure_name,metric_name,sex,val,upper,lower
0,164,642,163,6,1,3,2017,cong_neural,Birth,Neural tube defects,False,India,admin0,Incidence,Number,Both,18104.661515,20294.940574,16063.067351
1,164,642,179,6,1,3,2017,cong_neural,Birth,Neural tube defects,False,Ethiopia,admin0,Incidence,Number,Both,6828.132216,7838.261395,5942.329823
2,164,642,214,6,1,3,2017,cong_neural,Birth,Neural tube defects,False,Nigeria,admin0,Incidence,Number,Both,20365.078958,23229.228469,17743.129460
3,164,642,163,6,3,3,2017,cong_neural,Birth,Neural tube defects,False,India,admin0,Incidence,Rate,Both,0.000737,0.000826,0.000654
4,164,642,179,6,3,3,2017,cong_neural,Birth,Neural tube defects,False,Ethiopia,admin0,Incidence,Rate,Both,0.001838,0.002110,0.001600
5,164,642,214,6,3,3,2017,cong_neural,Birth,Neural tube defects,False,Nigeria,admin0,Incidence,Rate,Both,0.002611,0.002979,0.002275


In [15]:
ntd_birth_prev = bp[columns_to_keep]
ntd_birth_prev

,cause_name,location_name,age_group_name,measure_name,metric_name,val,upper,lower
0,Neural tube defects,India,Birth,Incidence,Number,18104.661515,20294.940574,16063.067351
1,Neural tube defects,Ethiopia,Birth,Incidence,Number,6828.132216,7838.261395,5942.329823
2,Neural tube defects,Nigeria,Birth,Incidence,Number,20365.078958,23229.228469,17743.129460
3,Neural tube defects,India,Birth,Incidence,Rate,0.000737,0.000826,0.000654
4,Neural tube defects,Ethiopia,Birth,Incidence,Rate,0.001838,0.002110,0.001600
5,Neural tube defects,Nigeria,Birth,Incidence,Rate,0.002611,0.002979,0.002275


In [16]:
ntd_birth_prev.shape

(6, 8)

### See if Jupyter will display un-rounded numbers so I can manually copy them into my spreadsheet

Yes, but only if I access them individually. Ugh, I'll just save the DataFrame as a .csv file and transfer it to my computer.

In [17]:
ntd_birth_prev.loc[3,'val']

0.000736894501707

In [18]:
ntd_birth_prev.loc[3:5, 'val']

3    0.000737
4    0.001838
5    0.002611
Name: val, dtype: float64

In [19]:
100_000*ntd_birth_prev.loc[3:5, 'val']

3     73.689450
4    183.833644
5    261.141493
Name: val, dtype: float64

# 4. Concatenate DALY and birth prevalence data and save file 

In [20]:
ntd_data = pd.concat([ntd_birth_prev, ntd_dalys], ignore_index=True)
print(ntd_data.shape)
ntd_data.head(10)

(36, 8)


,cause_name,location_name,age_group_name,measure_name,metric_name,val,upper,lower
0,Neural tube defects,India,Birth,Incidence,Number,18104.661515,20294.940574,16063.067351
1,Neural tube defects,Ethiopia,Birth,Incidence,Number,6828.132216,7838.261395,5942.329823
2,Neural tube defects,Nigeria,Birth,Incidence,Number,20365.078958,23229.228469,17743.129460
3,Neural tube defects,India,Birth,Incidence,Rate,0.000737,0.000826,0.000654
4,Neural tube defects,Ethiopia,Birth,Incidence,Rate,0.001838,0.002110,0.001600
5,Neural tube defects,Nigeria,Birth,Incidence,Rate,0.002611,0.002979,0.002275
6,Neural tube defects,India,Under 5,DALYs (Disability-Adjusted Life Years),Number,44936.678983,61226.307877,32247.186071
7,Neural tube defects,India,Early Neonatal,DALYs (Disability-Adjusted Life Years),Number,14185.414021,23900.946935,8044.466831
8,Neural tube defects,India,Late Neonatal,DALYs (Disability-Adjusted Life Years),Number,5392.882027,8773.937460,3530.200498
9,Neural tube defects,India,Post Neonatal,DALYs (Disability-Adjusted Life Years),Number,8478.122371,12004.392068,6376.735792


In [21]:
filename = 'neural_tube_defects_output_gbd2017.csv'
ntd_data.to_csv(filename, index=False)
pd.read_csv(filename).head(10) # Check that output looks the same when we read it in

,cause_name,location_name,age_group_name,measure_name,metric_name,val,upper,lower
0,Neural tube defects,India,Birth,Incidence,Number,18104.661515,20294.940574,16063.067351
1,Neural tube defects,Ethiopia,Birth,Incidence,Number,6828.132216,7838.261395,5942.329823
2,Neural tube defects,Nigeria,Birth,Incidence,Number,20365.078958,23229.228469,17743.129460
3,Neural tube defects,India,Birth,Incidence,Rate,0.000737,0.000826,0.000654
4,Neural tube defects,Ethiopia,Birth,Incidence,Rate,0.001838,0.002110,0.001600
5,Neural tube defects,Nigeria,Birth,Incidence,Rate,0.002611,0.002979,0.002275
6,Neural tube defects,India,Under 5,DALYs (Disability-Adjusted Life Years),Number,44936.678983,61226.307877,32247.186071
7,Neural tube defects,India,Early Neonatal,DALYs (Disability-Adjusted Life Years),Number,14185.414021,23900.946935,8044.466831
8,Neural tube defects,India,Late Neonatal,DALYs (Disability-Adjusted Life Years),Number,5392.882027,8773.937460,3530.200498
9,Neural tube defects,India,Post Neonatal,DALYs (Disability-Adjusted Life Years),Number,8478.122371,12004.392068,6376.735792


# 5. See what happens if we request DALY and birth prevalence data together

We get more rows, but they're all either NaN, 0, or redundant... see below

In [22]:
df = get_outputs(
    'cause',
    cause_id = list_ids('cause', 'Neural tube defects'),
    measure_id = list_ids('measure', 'DALYs (Disability-Adjusted Life Years)', 'Incidence'),
    metric_id = list_ids('metric', 'Number', 'Rate'),
    location_id = list_ids('location', 'India', 'Ethiopia', 'Nigeria'),
    age_group_id = list_ids('age_group', 
                            'Birth', 'Early Neonatal', 'Late Neonatal',
                            'Post Neonatal', '1 to 4', 'Under 5'),
    gbd_round_id = list_ids('gbd_round', '2017'),
    year_id = 2017,
)
print(df.shape)
df.head()

(72, 19)


,age_group_id,cause_id,location_id,measure_id,metric_id,sex_id,year_id,acause,age_group_name,cause_name,expected,location_name,location_type,measure_name,metric_name,sex,val,upper,lower
0,1,642,163,2,1,3,2017,cong_neural,Under 5,Neural tube defects,False,India,admin0,DALYs (Disability-Adjusted Life Years),Number,Both,44936.678983,61226.307877,32247.186071
1,2,642,163,2,1,3,2017,cong_neural,Early Neonatal,Neural tube defects,False,India,admin0,DALYs (Disability-Adjusted Life Years),Number,Both,14185.414021,23900.946935,8044.466831
2,3,642,163,2,1,3,2017,cong_neural,Late Neonatal,Neural tube defects,False,India,admin0,DALYs (Disability-Adjusted Life Years),Number,Both,5392.882027,8773.937460,3530.200498
3,4,642,163,2,1,3,2017,cong_neural,Post Neonatal,Neural tube defects,False,India,admin0,DALYs (Disability-Adjusted Life Years),Number,Both,8478.122371,12004.392068,6376.735792
4,5,642,163,2,1,3,2017,cong_neural,1 to 4,Neural tube defects,False,India,admin0,DALYs (Disability-Adjusted Life Years),Number,Both,16880.260564,22452.841899,11706.275501


In [23]:
# num rows = 2 measures x 2 metrics x 3 locations x 6 age groups
2*2*3*6

72

In [24]:
# There are probably missing values...
df.query('val != val')

,age_group_id,cause_id,location_id,measure_id,metric_id,sex_id,year_id,acause,age_group_name,cause_name,expected,location_name,location_type,measure_name,metric_name,sex,val,upper,lower
5,164,642,163,2,1,3,2017,cong_neural,Birth,Neural tube defects,NaN,India,admin0,DALYs (Disability-Adjusted Life Years),Number,Both,NaN,NaN,NaN
11,164,642,179,2,1,3,2017,cong_neural,Birth,Neural tube defects,NaN,Ethiopia,admin0,DALYs (Disability-Adjusted Life Years),Number,Both,NaN,NaN,NaN
17,164,642,214,2,1,3,2017,cong_neural,Birth,Neural tube defects,NaN,Nigeria,admin0,DALYs (Disability-Adjusted Life Years),Number,Both,NaN,NaN,NaN
31,2,642,214,6,1,3,2017,cong_neural,Early Neonatal,Neural tube defects,NaN,Nigeria,admin0,Incidence,Number,Both,NaN,NaN,NaN
32,3,642,214,6,1,3,2017,cong_neural,Late Neonatal,Neural tube defects,NaN,Nigeria,admin0,Incidence,Number,Both,NaN,NaN,NaN
33,4,642,214,6,1,3,2017,cong_neural,Post Neonatal,Neural tube defects,NaN,Nigeria,admin0,Incidence,Number,Both,NaN,NaN,NaN
34,5,642,214,6,1,3,2017,cong_neural,1 to 4,Neural tube defects,NaN,Nigeria,admin0,Incidence,Number,Both,NaN,NaN,NaN
41,164,642,163,2,3,3,2017,cong_neural,Birth,Neural tube defects,NaN,India,admin0,DALYs (Disability-Adjusted Life Years),Rate,Both,NaN,NaN,NaN
47,164,642,179,2,3,3,2017,cong_neural,Birth,Neural tube defects,NaN,Ethiopia,admin0,DALYs (Disability-Adjusted Life Years),Rate,Both,NaN,NaN,NaN
53,164,642,214,2,3,3,2017,cong_neural,Birth,Neural tube defects,NaN,Nigeria,admin0,DALYs (Disability-Adjusted Life Years),Rate,Both,NaN,NaN,NaN


In [25]:
# How many values are NaN?
df.query('val != val').shape

(14, 19)

In [26]:
df.dropna(inplace=True)
df.shape

(58, 19)

In [27]:
# How many values are 0? 
df.query('val == 0').shape

(16, 19)

In [28]:
df = df[df['val']>0]
df.shape

(42, 19)

There were 36 rows in our concatenated dataframe above, so there are 42-36=6 extras remaining after filtering out NaN's and 0's.

The 6 extra rows are for Under 5 incidence, Number and Rate. The Number for Under 5 is the same as the Number for Birth, so the only new data is the incidence rates in the Under 5 age group, which we don't need, but we could compute from the other data if we wanted.

In [29]:
df.query('age_group_name == "Under 5" and measure_name == "Incidence"')

,age_group_id,cause_id,location_id,measure_id,metric_id,sex_id,year_id,acause,age_group_name,cause_name,expected,location_name,location_type,measure_name,metric_name,sex,val,upper,lower
18,1,642,163,6,1,3,2017,cong_neural,Under 5,Neural tube defects,False,India,admin0,Incidence,Number,Both,18104.661515,20294.940574,16063.067351
24,1,642,179,6,1,3,2017,cong_neural,Under 5,Neural tube defects,False,Ethiopia,admin0,Incidence,Number,Both,6828.132216,7838.261395,5942.329823
30,1,642,214,6,1,3,2017,cong_neural,Under 5,Neural tube defects,False,Nigeria,admin0,Incidence,Number,Both,20365.078958,23229.228469,17743.129460
54,1,642,163,6,3,3,2017,cong_neural,Under 5,Neural tube defects,False,India,admin0,Incidence,Rate,Both,0.000140,0.000157,0.000124
60,1,642,179,6,3,3,2017,cong_neural,Under 5,Neural tube defects,False,Ethiopia,admin0,Incidence,Rate,Both,0.000411,0.000471,0.000357
66,1,642,214,6,3,3,2017,cong_neural,Under 5,Neural tube defects,False,Nigeria,admin0,Incidence,Rate,Both,0.000591,0.000674,0.000515
